In [4]:
# Centralized sonar defaults (inserted by sweep)
from utils.sonar_config import SONAR_VIS_DEFAULTS
from pathlib import Path
import utils.sonar_image_analysis as iau
from utils.sonar_config import IMAGE_PROCESSING_CONFIG
sonar_config = SONAR_VIS_DEFAULTS.copy()
# Backwards-compatible variable names used in older notebooks
RANGE_MIN_M = sonar_config['range_min_m']
RANGE_MAX_M = sonar_config['range_max_m']
DISPLAY_RANGE_MAX_M = sonar_config['display_range_max_m']

# Prefer selecting by bag ID (part of the NPZ filename) instead of a numeric index.
# Set TARGET_BAG to a substring that appears in the NPZ filename (e.g. '2024-08-22_14-29-05')
# '2024-08-22_14-47-39'
# '2024-08-22_14-29-05'
# '2024-08-20_14-31-29' HARD
TARGET_BAG = '2024-08-20_14-31-29'  # change this to your desired bag ID
from utils.sonar_config import EXPORTS_DIR_DEFAULT, EXPORTS_SUBDIRS
EXPORTS_FOLDER = Path(EXPORTS_DIR_DEFAULT)

# If you keep your raw .bag files or raw data on an external drive, point DATA_DIR there.
# Example (your external disk): /Volumes/LaCie/SOLAQUA/raw_data
DATA_DIR = Path("/Volumes/LaCie/SOLAQUA/raw_data")
print(f"Using DATA_DIR = {DATA_DIR}")

# Find NPZ files and pick the one matching TARGET_BAG
files = iau.get_available_npz_files()
if not files:
    raise FileNotFoundError(f"No NPZ files found in configured exports outputs (looked under {EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('outputs','outputs')})")
matches = [p for p in files if TARGET_BAG in p.name]
if not matches:
    # Help the user by listing available NPZ files
    print(f'No NPZ file matched TARGET_BAG={TARGET_BAG!r}')
    print('Available NPZ files:')
    for i,p in enumerate(files):
        print(f'  {i}: {p.name}')
    raise ValueError(f'No NPZ file contains "{TARGET_BAG}" in its name')

# If multiple matches, choose the most recently modified one
selected = max(matches, key=lambda p: p.stat().st_mtime)
NPZ_FILE_INDEX = files.index(selected)
print(f'Selected NPZ file: {selected.name} (index={NPZ_FILE_INDEX})')

Using DATA_DIR = /Volumes/LaCie/SOLAQUA/raw_data
Selected NPZ file: 2024-08-20_14-31-29_data_cones.npz (index=8)


# 🔍 Net Detection Pipeline Visualization

This section creates individual figures to visualize each step in the net detection pipeline. Each step shows the transformation of the sonar image as it progresses through the processing stages.

## Pipeline Overview:
1. **Original Frame** - Raw sonar data (uint8 grayscale)
2. **Momentum Enhancement** - Directional momentum merge for sharper objects  
3. **Edge Detection** - Canny edge detection on enhanced image
4. **Edge Processing** - Morphological operations (close + dilate)
5. **Contour Detection** - Find contours from processed edges
6. **Contour Selection** - Select best contour based on area, elongation, and tracking
7. **Final Result** - Distance measurement and AOI tracking

# Simple Image Analysis with CV2

This notebook demonstrates:
1. **Pick a frame** from NPZ files and save it locally
2. **Use standard cv2 functions** directly for image processing
3. **Experiment** with different OpenCV operations

## Distance Analysis Over Time

Now let's perform a comprehensive analysis of the red line distance over time. The red line represents the major axis of the detected elongated contour (likely a fishing net), and we'll track how this distance changes throughout the video sequence.

In [5]:
# Create video using the CORE simplified processor with ELLIPTICAL AOI
# This will now show the yellow ELLIPTICAL AOI instead of rectangular!

# Create the unified processor
processor = iau.SonarDataProcessor()

# Use the enhanced video creation with CORE processor
video_path = iau.create_enhanced_contour_detection_video_with_processor(
    npz_file_index=NPZ_FILE_INDEX,          # Which NPZ file to use
    frame_start=1,           # Starting frame
    frame_count=1500,         # Start with 1500 frames for testing
    frame_step=1,            # Step between frames
    output_path=Path(EXPORTS_DIR_DEFAULT) / EXPORTS_SUBDIRS.get('videos','videos') / 'elliptical_aoi_tracking.mp4',
    processor=processor      # Use our CORE processor with ellipse tracking
)

print(f"✅ Video created with ELLIPTICAL AOI visualization!")
print(f"🎯 The AOI should now appear as a yellow ELLIPSE that smoothly tracks the net")
print(f"📁 Video saved to: {video_path}")

=== ENHANCED ELLIPTICAL AOI VIDEO CREATION ===
Creating video with elliptical AOI tracking...
Frames: 1500, step: 1
✅ Processing 1059 frames with elliptical AOI...
✅ Processing 1059 frames with elliptical AOI...
Processed 110/1059 frames | Ellipse tracking: 0
Processed 110/1059 frames | Ellipse tracking: 0
Processed 120/1059 frames | Ellipse tracking: 0
Processed 120/1059 frames | Ellipse tracking: 0
Processed 130/1059 frames | Ellipse tracking: 0
Processed 130/1059 frames | Ellipse tracking: 0
Processed 150/1059 frames | Ellipse tracking: 0
Processed 150/1059 frames | Ellipse tracking: 0
Processed 190/1059 frames | Ellipse tracking: 0
Processed 190/1059 frames | Ellipse tracking: 0
Processed 380/1059 frames | Ellipse tracking: 0
Processed 380/1059 frames | Ellipse tracking: 0
Processed 390/1059 frames | Ellipse tracking: 0
Processed 390/1059 frames | Ellipse tracking: 0
Processed 420/1059 frames | Ellipse tracking: 0
Processed 420/1059 frames | Ellipse tracking: 0
Processed 440/1059 f

In [6]:
# Use the new Distance Analysis Engine with elliptical AOI
engine = iau.DistanceAnalysisEngine()
distance_results = engine.analyze_npz_sequence(
    npz_file_index=NPZ_FILE_INDEX,    
    frame_start=1,        # Start from frame 1
    frame_count=1500,      # Analyze all frames from the video
    frame_step=1          # Every frame
)

=== DISTANCE ANALYSIS FROM NPZ ===
Analyzing: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_14-31-29_data_cones.npz
Processing 1059 frames from 1 (step=1)
Processing 1059 frames from 1 (step=1)
  Processed 50/1059 frames (Success rate: 100.0%)
  Processed 50/1059 frames (Success rate: 100.0%)
  Processed 100/1059 frames (Success rate: 100.0%)
  Processed 100/1059 frames (Success rate: 100.0%)
  Processed 150/1059 frames (Success rate: 100.0%)
  Processed 150/1059 frames (Success rate: 100.0%)
  Processed 200/1059 frames (Success rate: 100.0%)
  Processed 200/1059 frames (Success rate: 100.0%)
  Processed 250/1059 frames (Success rate: 100.0%)
  Processed 250/1059 frames (Success rate: 100.0%)
  Processed 300/1059 frames (Success rate: 100.0%)
  Processed 300/1059 frames (Success rate: 100.0%)
  Processed 350/1059 frames (Success rate: 100.0%)
  Processed 350/1059 frames (Success rate: 100.0%)
  Processed 400/1059 frames (Success rate: 100.0%)
  Processed 400/1059 frames (Success ra

## Convert to Real-World Distances

Now let's convert the pixel distances to real-world distances using the fact that the entire sonar image represents a 10x10 meter area.

In [7]:
# Auto-detect pixel->meter mapping from the selected NPZ (if available)
try:
    cones, ts, extent, meta = iau.load_cone_run_npz(selected)
    T, H, W = cones.shape
    x_min, x_max, y_min, y_max = extent
    width_m = float(x_max - x_min)
    height_m = float(y_max - y_min)
    px2m_x = width_m / float(W)
    px2m_y = height_m / float(H)
    pixels_to_meters_avg = 0.5 * (px2m_x + px2m_y)
    image_shape = (H, W)
    sonar_coverage_meters = max(width_m, height_m)
    print(f"Detected NPZ extent: x=[{x_min:.3f},{x_max:.3f}] m, y=[{y_min:.3f},{y_max:.3f}] m")
    print(f"Image shape from NPZ: H={H}, W={W}")
    print(f"meters/pixel: x={px2m_x:.6f}, y={px2m_y:.6f}, avg={pixels_to_meters_avg:.6f}")
except Exception as e:
    print("Could not read NPZ metadata:", e)
    print("Falling back to defaults from sonar_config.")
    from utils.sonar_config import CONE_H_DEFAULT, CONE_W_DEFAULT, DISPLAY_RANGE_MAX_M_DEFAULT
    image_shape = (CONE_H_DEFAULT, CONE_W_DEFAULT)
    sonar_coverage_meters = DISPLAY_RANGE_MAX_M_DEFAULT * 2  # approximate
    pixels_to_meters_avg = sonar_coverage_meters / max(image_shape)
    
# Use the computed pixels_to_meters_avg in downstream analysis
print(f"Using pixels_to_meters_avg = {pixels_to_meters_avg:.6f} m/px")

Detected NPZ extent: x=[-8.660,8.660] m, y=[0.000,10.000] m
Image shape from NPZ: H=700, W=900
meters/pixel: x=0.019245, y=0.014286, avg=0.016765
Using pixels_to_meters_avg = 0.016765 m/px


In [8]:
# Real-world distance analysis and plotting using utility function
iau.plot_real_world_distance_analysis(distance_results, image_shape=image_shape, sonar_coverage_meters=sonar_coverage_meters)

In [9]:
# 🔄 COMPARISON: SONAR vs DVL DISTANCE MEASUREMENTS
# =================================================
import utils.net_distance_analysis as sda

print(f"🎯 LOADING DVL DATA FOR COMPARISON: {TARGET_BAG}")
print("=" * 60)

# IMPORTANT: Pass the by_bag folder, not just the exports root
# The function expects the folder containing the CSV files
from utils.sonar_config import EXPORTS_SUBDIRS
BY_BAG_FOLDER = EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('by_bag', 'by_bag')

# Load all distance data for the target bag
raw_data, distance_measurements = sda.load_all_distance_data_for_bag(TARGET_BAG, BY_BAG_FOLDER)

# Display what we loaded
print(f"\n📊 RAW DATA LOADED:")
for key, data in raw_data.items():
    if data is not None:
        print(f"   ✅ {key}: {len(data)} records")
    else:
        print(f"   ❌ {key}: None")

print(f"\n📏 DISTANCE MEASUREMENTS LOADED:")
for key, info in distance_measurements.items():
    data_len = len(info['data'])
    print(f"   ✅ {key}: {data_len} records - {info['description']}")

try:
    fig, comparison_stats = iau.interactive_distance_comparison(distance_results, raw_data, sonar_coverage_m=sonar_coverage_meters, sonar_image_size=image_shape[0])
    if comparison_stats and isinstance(comparison_stats, dict) and 'error' in comparison_stats:
        print('Comparison did not run:', comparison_stats['error'])
    else:
        print('\nComparison stats summary:')
        from pprint import pprint
        pprint(comparison_stats)
        # Display the interactive plot
        fig.show()
except Exception as e:
    print('Error running interactive comparison:', e)

🎯 LOADING DVL DATA FOR COMPARISON: 2024-08-20_14-31-29
🎯 LOADING ALL DISTANCE DATA FOR BAG: 2024-08-20_14-31-29
📡 1. Loading Navigation Data...
   ✅ Loaded 559 navigation records
📡 2. Loading Guidance Data...
   ✅ Loaded 549 guidance records with ['error_net_distance', 'desired_net_distance', 'r_net_distance_d']
📡 3. Loading DVL Altimeter...
   ❌ DVL altimeter file not found
📡 4. Loading USBL...
   ✅ Loaded 33 USBL records
📡 5. Loading DVL Position...
   ✅ Loaded 283 DVL position records
📡 6. Loading Navigation Position...
   ✅ Loaded 278 navigation position records
📡 7. Loading INS Z Position...
   ❌ INS file not found

📊 LOADING SUMMARY:
   🎯 Target bag: 2024-08-20_14-31-29
   📁 Raw data loaded: 2/2
   📏 Distance measurements: 4

📊 RAW DATA LOADED:
   ✅ navigation: 559 records
   ✅ guidance: 549 records

📏 DISTANCE MEASUREMENTS LOADED:
   ✅ USBL_3D: 33 records - 3D acoustic position
   ✅ USBL_Depth: 33 records - USBL depth measurement
   ✅ DVL_Position: 283 records - 3D DVL position


/Users/eirikvarnes/code/SOLAQUA/utils/sonar_image_analysis.py:505: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/Users/eirikvarnes/code/SOLAQUA/utils/sonar_image_analysis.py:512: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.




📊 SONAR vs DVL COMPARISON STATISTICS:
Sonar mean distance: 2.085 m
DVL mean distance:   1.968 m
Scale ratio (Sonar/DVL): 1.060x
Sonar duration: 67.1s (1059 frames)
DVL duration:   67.0s (559 records)

Comparison stats summary:
{'dvl_duration_s': 67.007197142,
 'dvl_mean_m': 1.9676565289337435,
 'dvl_records': 559,
 'scale_ratio': 1.0595705518465377,
 'sonar_duration_s': 67.07053097672778,
 'sonar_frames': 1059,
 'sonar_mean_m': 2.0848709142067694}


In [10]:
# 🎥 Generate Video with Both DVL and Sonar Analysis Overlays
# ========================================================
import utils.sonar_and_foto_generation as sg

# Set up paths
VIDEO_SEQ_DIR = None  # Set to path if you want camera footage included

print("🎬 GENERATING VIDEO WITH DUAL NET DISTANCE OVERLAYS")
print("=" * 60)
print(f"🎯 Target Bag: {TARGET_BAG}")
print(f"📊 DVL Data: {len(raw_data.get('navigation', []))} records")
print(f"📏 Sonar Analysis: {len(distance_results)} frames")
print(f"🎥 Camera: {'enabled' if VIDEO_SEQ_DIR else 'disabled'}")

# Generate the video with both overlays
try:
    video_path = sg.export_optimized_sonar_video(
        TARGET_BAG=TARGET_BAG,
        EXPORTS_FOLDER=EXPORTS_FOLDER,
        START_IDX=1,
        END_IDX=1000,  # Adjust as needed
        STRIDE=1,
        VIDEO_SEQ_DIR=VIDEO_SEQ_DIR,
        INCLUDE_NET=True,  # Enable DVL net distance overlay
        SONAR_DISTANCE_RESULTS=distance_results,  # Enable sonar analysis overlay
        NET_DISTANCE_TOLERANCE=0.5,
        NET_PITCH_TOLERANCE=2.0,  # Increased from 0.3 to 2.0 seconds for better pitch sync
    )
    print(f"\n✅ Video generated successfully!")
    print(f"📁 Output: {video_path}")
except Exception as e:
    print(f"❌ Error generating video: {e}")

🎬 GENERATING VIDEO WITH DUAL NET DISTANCE OVERLAYS
🎯 Target Bag: 2024-08-20_14-31-29
📊 DVL Data: 559 records
📏 Sonar Analysis: 1059 frames
🎥 Camera: disabled
🛠️ OPTIMIZED SONAR VIDEO
🎯 Target Bag: 2024-08-20_14-31-29
   Cone Size: 900x700
   Range: 0.0-5.0m | FOV: 120.0°
   🎥 Camera: disabled
   🕸  Net-line: enabled (dist tol=0.5s, pitch tol=2.0s)
   📊 Sonar Analysis: enabled
   Loading sonar data: sensor_sonoptix_echo_image__2024-08-20_14-31-29_video.csv
   ✅ Loaded 1060 sonar frames in 42.49s
   ✅ Loaded 559 navigation records in 0.01s
      Available: ['NetDistance', 'NetPitch', 'timestamp']
   Frames: 1..999 (step 1) => 999
   Natural FPS: 15.7
   ✅ Loaded 1060 sonar frames in 42.49s
   ✅ Loaded 559 navigation records in 0.01s
      Available: ['NetDistance', 'NetPitch', 'timestamp']
   Frames: 1..999 (step 1) => 999
   Natural FPS: 15.7
❌ Frame 580 error: cannot access local variable 'sonar_label' where it is not associated with a value
❌ Frame 580 error: cannot access local varia

KeyboardInterrupt: 

## 🎯 Unified Masking System

New integrated pixel-level masking system for preventing unwanted object merging:

In [ ]:
# 🎯 UNIFIED MASKING SYSTEM DEMO
# ===============================
# Demonstrate the new SonarMaskingSystem for preventing unwanted object merging

from utils.sonar_image_analysis import SonarMaskingSystem, preprocess_edges_with_masking
import numpy as np
import matplotlib.pyplot as plt
import cv2

print("🎯 UNIFIED MASKING SYSTEM DEMONSTRATION")
print("=" * 60)

if 'cones' in locals() and len(cones) > 0:
    # Get a test frame
    test_frame = cones[150].astype(np.uint8)
    frame_shape = test_frame.shape
    
    print(f"📏 Frame shape: {frame_shape}")
    
    # === CONFIGURATION EXAMPLE ===
    print("\n⚙️  MASKING SYSTEM CONFIGURATION:")
    
    # Example 1: Simple static exclusion zone
    masking_config_static = IMAGE_PROCESSING_CONFIG.copy()
    masking_config_static['masking_system'] = {
        'enabled': True,
        'mode': 'static',
        'static_zones': [
            {'center': (200, 300), 'radius': 50, 'shape': 'circle'},
            {'center': (400, 200), 'radius': 30, 'shape': 'circle'}
        ],
        'apply_to_enhancement': True,
        'apply_to_edge_detection': True,
    }
    
    # Example 2: Dynamic object-based exclusions
    masking_config_dynamic = IMAGE_PROCESSING_CONFIG.copy()
    masking_config_dynamic['masking_system'] = {
        'enabled': True,
        'mode': 'dynamic',
        'dynamic_exclusions': {
            'min_object_area': 100,
            'exclusion_radius': 15,
            'radius_scale_factor': 1.5,
            'max_zones': 5,
            'zone_lifetime_frames': 3,
        },
        'mask_application': {
            'feather_radius': 5,
            'gradient_falloff': True,
            'preserve_thin_structures': True,
        },
        'apply_to_enhancement': True,
        'apply_to_edge_detection': True,
    }
    
    # === CREATE MASKING SYSTEMS ===
    print("🔧 Creating masking system instances...")
    
    # No masking (baseline)
    no_masking = None
    
    # Static exclusion zones
    static_masking = SonarMaskingSystem(frame_shape, masking_config_static)
    
    # Dynamic masking (simulate detected objects)
    dynamic_masking = SonarMaskingSystem(frame_shape, masking_config_dynamic)
    
    # Simulate some detected objects for dynamic masking
    simulated_objects = [
        {'center': (150, 250), 'area': 200, 'id': 1},
        {'center': (350, 180), 'area': 150, 'id': 2},
        {'center': (180, 400), 'area': 300, 'id': 3},
    ]
    dynamic_masking.update_dynamic_exclusions(simulated_objects)
    
    # === PROCESS WITH DIFFERENT MASKING MODES ===
    print("🔄 Processing with different masking modes...")
    
    results = {}
    
    # 1. No masking (baseline)
    no_mask_edges = preprocess_edges_with_masking(test_frame, no_masking, IMAGE_PROCESSING_CONFIG)
    results['No Masking'] = {
        'edges': no_mask_edges[1],
        'pixels': np.count_nonzero(no_mask_edges[1])
    }
    
    # 2. Static masking
    static_mask = static_masking.generate_mask()
    static_edges = preprocess_edges_with_masking(test_frame, static_masking, masking_config_static)
    results['Static Masking'] = {
        'edges': static_edges[1],
        'mask': static_mask,
        'pixels': np.count_nonzero(static_edges[1])
    }
    
    # 3. Dynamic masking
    dynamic_mask = dynamic_masking.generate_mask()
    dynamic_edges = preprocess_edges_with_masking(test_frame, dynamic_masking, masking_config_dynamic)
    results['Dynamic Masking'] = {
        'edges': dynamic_edges[1],
        'mask': dynamic_mask,
        'pixels': np.count_nonzero(dynamic_edges[1])
    }
    
    # === VISUALIZATION ===
    print("📊 Results comparison:")
    for mode, result in results.items():
        print(f"  {mode:15s}: {result['pixels']:,} edge pixels")
    
    # Create comprehensive visualization
    fig, axes = plt.subplots(3, 3, figsize=(18, 18))
    
    # Row 1: Original and masks
    axes[0, 0].imshow(test_frame, cmap='gray', aspect='equal')
    axes[0, 0].set_title('Original Frame', fontsize=14)
    axes[0, 0].set_xlabel('X (pixels)')
    axes[0, 0].set_ylabel('Y (pixels)')
    
    axes[0, 1].imshow(results['Static Masking']['mask'], cmap='gray', aspect='equal')
    axes[0, 1].set_title('Static Exclusion Mask', fontsize=14)
    axes[0, 1].set_xlabel('X (pixels)')
    axes[0, 1].set_ylabel('Y (pixels)')
    
    axes[0, 2].imshow(results['Dynamic Masking']['mask'], cmap='gray', aspect='equal')
    axes[0, 2].set_title('Dynamic Exclusion Mask', fontsize=14)
    axes[0, 2].set_xlabel('X (pixels)')
    axes[0, 2].set_ylabel('Y (pixels)')
    
    # Row 2: Edge detection results
    axes[1, 0].imshow(results['No Masking']['edges'], cmap='gray', aspect='equal')
    axes[1, 0].set_title(f'No Masking\\n{results[\"No Masking\"][\"pixels\"]:,} pixels', fontsize=14)
    axes[1, 0].set_xlabel('X (pixels)')
    axes[1, 0].set_ylabel('Y (pixels)')
    
    axes[1, 1].imshow(results['Static Masking']['edges'], cmap='gray', aspect='equal')
    axes[1, 1].set_title(f'Static Masking\\n{results[\"Static Masking\"][\"pixels\"]:,} pixels', fontsize=14)
    axes[1, 1].set_xlabel('X (pixels)')
    axes[1, 1].set_ylabel('Y (pixels)')
    
    axes[1, 2].imshow(results['Dynamic Masking']['edges'], cmap='gray', aspect='equal')
    axes[1, 2].set_title(f'Dynamic Masking\\n{results[\"Dynamic Masking\"][\"pixels\"]:,} pixels', fontsize=14)
    axes[1, 2].set_xlabel('X (pixels)')
    axes[1, 2].set_ylabel('Y (pixels)')
    
    # Row 3: Difference maps
    diff_static = cv2.absdiff(results['No Masking']['edges'], results['Static Masking']['edges'])
    diff_dynamic = cv2.absdiff(results['No Masking']['edges'], results['Dynamic Masking']['edges'])
    
    axes[2, 0].axis('off')  # Empty for layout
    
    axes[2, 1].imshow(diff_static, cmap='hot', aspect='equal')
    axes[2, 1].set_title('Static Masking\\nDifference from Baseline', fontsize=14)
    axes[2, 1].set_xlabel('X (pixels)')
    axes[2, 1].set_ylabel('Y (pixels)')
    
    axes[2, 2].imshow(diff_dynamic, cmap='hot', aspect='equal')
    axes[2, 2].set_title('Dynamic Masking\\nDifference from Baseline', fontsize=14)
    axes[2, 2].set_xlabel('X (pixels)')
    axes[2, 2].set_ylabel('Y (pixels)')
    
    plt.suptitle('Unified Masking System - Comparison of Masking Modes', fontsize=16, y=0.98)
    plt.tight_layout()
    plt.show()
    
    # === USAGE EXAMPLES ===
    print(f"\n💡 USAGE EXAMPLES:")
    print(f"   # 1. Enable static exclusion zones:")
    print(f"   IMAGE_PROCESSING_CONFIG['masking_system']['enabled'] = True")
    print(f"   IMAGE_PROCESSING_CONFIG['masking_system']['static_zones'] = [")
    print(f"       {{'center': (200, 300), 'radius': 50, 'shape': 'circle'}}")
    print(f"   ]")
    print(f"")
    print(f"   # 2. Enable dynamic object exclusions:")
    print(f"   IMAGE_PROCESSING_CONFIG['masking_system']['mode'] = 'dynamic'")
    print(f"   IMAGE_PROCESSING_CONFIG['masking_system']['dynamic_exclusions']['min_object_area'] = 150")
    print(f"")
    print(f"   # 3. Use in your pipeline:")
    print(f"   masking_system = SonarMaskingSystem(frame.shape, IMAGE_PROCESSING_CONFIG)")
    print(f"   edges = preprocess_edges_with_masking(frame, masking_system)")
    
    print(f"\n🎯 The new masking system provides:")
    print(f"   ✅ Unified configuration in IMAGE_PROCESSING_CONFIG['masking_system']")
    print(f"   ✅ Multiple masking modes: static, dynamic, adaptive, ownership")
    print(f"   ✅ Integration at all pipeline stages")
    print(f"   ✅ Backward compatibility with legacy functions")
    print(f"   ✅ Advanced features: feathering, gradient falloff, structure preservation")
    
else:
    print("❌ No sonar data available. Load sonar data first.")
    print("💡 Run the data loading cells above to get 'cones' variable.")

SyntaxError: unexpected character after line continuation character (3460883378.py, line 133)

## 🎯 AOI-Aware Masking

Test the new Area of Interest (AOI) aware masking logic:

In [ ]:
# 🎯 AOI PIXEL CLIPPING DEMONSTRATION
# ===================================
# Show how AOI pixel clipping ensures only pixels inside AOI are processed

from utils.sonar_image_analysis import SonarMaskingSystem, preprocess_edges_with_masking
import numpy as np
import matplotlib.pyplot as plt
import cv2

print("🎯 AOI PIXEL CLIPPING DEMONSTRATION")
print("=" * 60)

if 'cones' in locals() and len(cones) > 0:
    # Get a test frame
    test_frame = cones[120].astype(np.uint8)
    frame_shape = test_frame.shape
    
    print(f"📏 Frame shape: {frame_shape}")
    
    # === CONFIGURATION COMPARISON ===
    print("⚙️  Testing different AOI clipping modes:")
    
    # Mode 1: No AOI clipping (standard processing)
    config_no_aoi = IMAGE_PROCESSING_CONFIG.copy()
    config_no_aoi['masking_system'] = {
        'enabled': False,  # Disable all masking
    }
    
    # Mode 2: AOI with flexible boundary (allows processing outside AOI)
    config_flexible_aoi = IMAGE_PROCESSING_CONFIG.copy()
    config_flexible_aoi['masking_system'] = {
        'enabled': True,
        'mode': 'adaptive',
        'aoi_integration': {
            'enabled': True,
            'aoi_pixel_clipping': False,      # Allow processing outside AOI
            'aoi_overrides_exclusions': True,
            'strict_aoi_boundary': False,     # Allow feathering across boundary
        },
        'apply_to_edge_detection': True,
    }
    
    # Mode 3: AOI with strict pixel clipping (ONLY AOI pixels processed)
    config_strict_aoi = IMAGE_PROCESSING_CONFIG.copy()
    config_strict_aoi['masking_system'] = {
        'enabled': True,
        'mode': 'adaptive',
        'aoi_integration': {
            'enabled': True,
            'aoi_pixel_clipping': True,       # ONLY process pixels inside AOI
            'aoi_overrides_exclusions': True,
            'strict_aoi_boundary': True,      # Hard boundary at AOI edge
            'prevent_exclusions_in_aoi': True,
        },
        'apply_to_edge_detection': True,
    }
    
    # === CREATE MASKING SYSTEMS ===
    print("🔧 Creating masking systems with different AOI modes...")
    
    # Define a representative AOI (simulating detected net area)
    H, W = frame_shape
    aoi_center = (W // 2, H // 2 + 50)  # Slightly below center
    aoi_radius = min(W, H) // 4
    
    print(f"🎯 AOI: Center=({aoi_center[0]}, {aoi_center[1]}), Radius={aoi_radius}")
    
    # System 1: No AOI
    no_aoi_system = None
    
    # System 2: Flexible AOI
    flexible_aoi_system = SonarMaskingSystem(frame_shape, config_flexible_aoi)
    flexible_aoi_system.update_aoi({'type': 'circle', 'center': aoi_center, 'radius': aoi_radius})
    
    # System 3: Strict AOI clipping
    strict_aoi_system = SonarMaskingSystem(frame_shape, config_strict_aoi)
    strict_aoi_system.update_aoi({'type': 'circle', 'center': aoi_center, 'radius': aoi_radius})
    
    # === PROCESS WITH DIFFERENT MODES ===
    print("🔄 Processing with different AOI clipping modes...")
    
    results = {}
    
    # 1. No AOI (baseline)
    no_aoi_edges = preprocess_edges_with_masking(test_frame, no_aoi_system, config_no_aoi)
    results['No AOI'] = {
        'edges': no_aoi_edges[1],
        'pixels': np.count_nonzero(no_aoi_edges[1]),
        'mask': np.ones(frame_shape, dtype=np.uint8) * 255
    }
    
    # 2. Flexible AOI
    flexible_mask = flexible_aoi_system.generate_mask()
    flexible_edges = preprocess_edges_with_masking(test_frame, flexible_aoi_system, config_flexible_aoi)
    results['Flexible AOI'] = {
        'edges': flexible_edges[1],
        'mask': flexible_mask,
        'pixels': np.count_nonzero(flexible_edges[1])
    }
    
    # 3. Strict AOI clipping
    strict_mask = strict_aoi_system.generate_mask()
    strict_edges = preprocess_edges_with_masking(test_frame, strict_aoi_system, config_strict_aoi)
    results['Strict AOI Clipping'] = {
        'edges': strict_edges[1],
        'mask': strict_mask,
        'pixels': np.count_nonzero(strict_edges[1])
    }
    
    # === ANALYSIS ===
    print("📊 Results comparison:")
    for mode, result in results.items():
        print(f"  {mode:20s}: {result['pixels']:,} edge pixels")
    
    # Calculate pixel reduction
    baseline_pixels = results['No AOI']['pixels']
    flexible_pixels = results['Flexible AOI']['pixels'] 
    strict_pixels = results['Strict AOI Clipping']['pixels']
    
    flexible_reduction = (baseline_pixels - flexible_pixels) / baseline_pixels * 100
    strict_reduction = (baseline_pixels - strict_pixels) / baseline_pixels * 100
    
    print(f"\n📉 Pixel reduction from baseline:")
    print(f"  Flexible AOI:     -{flexible_reduction:.1f}%")
    print(f"  Strict Clipping:  -{strict_reduction:.1f}%")
    
    # === VISUALIZATION ===
    fig, axes = plt.subplots(3, 3, figsize=(18, 18))
    
    # Row 1: Original and AOI masks
    axes[0, 0].imshow(test_frame, cmap='gray', aspect='equal')
    axes[0, 0].set_title('Original Frame', fontsize=14)
    axes[0, 0].set_xlabel('X (pixels)')
    axes[0, 0].set_ylabel('Y (pixels)')
    
    # Draw AOI circle on original
    circle = plt.Circle(aoi_center, aoi_radius, fill=False, color='red', linewidth=2)
    axes[0, 0].add_patch(circle)
    axes[0, 0].text(aoi_center[0], aoi_center[1] - aoi_radius - 20, 'AOI', 
                   ha='center', va='bottom', color='red', fontweight='bold')
    
    axes[0, 1].imshow(results['Flexible AOI']['mask'], cmap='gray', aspect='equal')
    axes[0, 1].set_title('Flexible AOI Mask', fontsize=14)
    axes[0, 1].set_xlabel('X (pixels)')
    axes[0, 1].set_ylabel('Y (pixels)')
    
    axes[0, 2].imshow(results['Strict AOI Clipping']['mask'], cmap='gray', aspect='equal')
    axes[0, 2].set_title('Strict AOI Clipping Mask', fontsize=14)
    axes[0, 2].set_xlabel('X (pixels)')
    axes[0, 2].set_ylabel('Y (pixels)')
    
    # Row 2: Edge detection results
    axes[1, 0].imshow(results['No AOI']['edges'], cmap='gray', aspect='equal')
    axes[1, 0].set_title(f'No AOI\\n{results[\"No AOI\"][\"pixels\"]:,} pixels', fontsize=14)
    axes[1, 0].set_xlabel('X (pixels)')
    axes[1, 0].set_ylabel('Y (pixels)')
    
    axes[1, 1].imshow(results['Flexible AOI']['edges'], cmap='gray', aspect='equal')
    axes[1, 1].set_title(f'Flexible AOI\\n{results[\"Flexible AOI\"][\"pixels\"]:,} pixels', fontsize=14)
    axes[1, 1].set_xlabel('X (pixels)')
    axes[1, 1].set_ylabel('Y (pixels)')
    
    axes[1, 2].imshow(results['Strict AOI Clipping']['edges'], cmap='gray', aspect='equal')
    axes[1, 2].set_title(f'Strict AOI Clipping\\n{results[\"Strict AOI Clipping\"][\"pixels\"]:,} pixels', fontsize=14)
    axes[1, 2].set_xlabel('X (pixels)')
    axes[1, 2].set_ylabel('Y (pixels)')
    
    # Row 3: Difference analysis
    diff_flexible = cv2.absdiff(results['No AOI']['edges'], results['Flexible AOI']['edges'])
    diff_strict = cv2.absdiff(results['No AOI']['edges'], results['Strict AOI Clipping']['edges'])
    overlap_analysis = cv2.bitwise_and(results['Flexible AOI']['edges'], results['Strict AOI Clipping']['edges'])
    
    axes[2, 0].imshow(diff_flexible, cmap='hot', aspect='equal')
    axes[2, 0].set_title('Flexible AOI\\nDifference from Baseline', fontsize=14)
    axes[2, 0].set_xlabel('X (pixels)')
    axes[2, 0].set_ylabel('Y (pixels)')
    
    axes[2, 1].imshow(diff_strict, cmap='hot', aspect='equal')
    axes[2, 1].set_title('Strict Clipping\\nDifference from Baseline', fontsize=14)
    axes[2, 1].set_xlabel('X (pixels)')
    axes[2, 1].set_ylabel('Y (pixels)')
    
    axes[2, 2].imshow(overlap_analysis, cmap='gray', aspect='equal')
    axes[2, 2].set_title('Overlap: Flexible ∩ Strict\\n(Common Detected Edges)', fontsize=14)
    axes[2, 2].set_xlabel('X (pixels)')
    axes[2, 2].set_ylabel('Y (pixels)')
    
    plt.suptitle('AOI Pixel Clipping - Impact on Edge Detection', fontsize=16, y=0.98)
    plt.tight_layout()
    plt.show()
    
    # === USAGE EXAMPLES ===
    print(f"\n💡 USAGE EXAMPLES:")
    print(f"   # Enable strict AOI pixel clipping:")
    print(f"   IMAGE_PROCESSING_CONFIG['masking_system']['aoi_integration']['aoi_pixel_clipping'] = True")
    print(f"   IMAGE_PROCESSING_CONFIG['masking_system']['aoi_integration']['strict_aoi_boundary'] = True")
    print(f"")
    print(f"   # Use in pipeline:")
    print(f"   masking_system = SonarMaskingSystem(frame.shape)")
    print(f"   masking_system.update_aoi({{'type': 'circle', 'center': net_center, 'radius': net_radius}})")
    print(f"   edges = preprocess_edges_with_masking(frame, masking_system)")
    
    print(f"\n🎯 Key Benefits of AOI Pixel Clipping:")
    print(f"   ✅ ONLY processes pixels inside the net area")
    print(f"   ✅ Prevents partial objects from contributing outside pixels")
    print(f"   ✅ Creates clean boundaries at AOI edges")
    print(f"   ✅ Reduces processing of irrelevant background")
    print(f"   ✅ Improves net contour purity")
    
    # Show the key difference
    clipped_pixel_reduction = strict_reduction - flexible_reduction
    print(f"\n📈 Additional pixel reduction from strict clipping: {clipped_pixel_reduction:.1f}%")
    print(f"   This represents pixels that would have been included from objects")
    print(f"   partially entering the AOI, but are now properly excluded.")
    
else:
    print("❌ No sonar data available. Load sonar data first.")
    print("💡 Run the data loading cells above to get 'cones' variable.")

## 🔧 Integration Fix Test

Test the fixed integration to ensure AOI pixel clipping works in the actual pipeline:

In [ ]:
# 🔧 INTEGRATION FIX TEST
# ========================
# Test that AOI pixel clipping is now working in the actual processing pipeline

print("🔧 TESTING FIXED AOI INTEGRATION")
print("=" * 60)

# Verify the masking system configuration
print("⚙️  Current masking system configuration:")
masking_config = IMAGE_PROCESSING_CONFIG.get('masking_system', {})
print(f"  Enabled: {masking_config.get('enabled', False)}")
print(f"  AOI Integration Enabled: {masking_config.get('aoi_integration', {}).get('enabled', False)}")
print(f"  AOI Pixel Clipping: {masking_config.get('aoi_integration', {}).get('aoi_pixel_clipping', False)}")
print(f"  Prevent Exclusions in AOI: {masking_config.get('aoi_integration', {}).get('prevent_exclusions_in_aoi', False)}")

if 'processor' in locals():
    print(f"\n🔍 Testing with existing processor...")
    
    # Reset the processor to ensure clean state
    processor.reset_tracking()
    
    # Test with a frame that has data
    if 'cones' in locals() and len(cones) > 100:
        test_frame = cones[100].astype(np.uint8)
        print(f"📊 Processing frame shape: {test_frame.shape}")
        
        # Process the frame
        try:
            result = processor.analyze_frame(test_frame)
            print(f"✅ Frame processed successfully!")
            print(f"   Distance: {result.distance_pixels:.1f} pixels")
            print(f"   Angle: {result.angle_degrees:.1f}°")
            
            # Check if masking system was created
            if hasattr(processor, 'masking_system') and processor.masking_system is not None:
                print(f"✅ Masking system initialized!")
                
                # Check if AOI was set
                if processor.masking_system.current_aoi is not None:
                    aoi = processor.masking_system.current_aoi
                    print(f"✅ AOI set: Type={aoi.get('type')}, Center={aoi.get('center')}")
                    
                    # Test mask generation
                    mask = processor.masking_system.generate_mask()
                    pixels_allowed = np.count_nonzero(mask)
                    total_pixels = mask.shape[0] * mask.shape[1]
                    percentage_allowed = (pixels_allowed / total_pixels) * 100
                    
                    print(f"📊 Mask statistics:")
                    print(f"   Total pixels: {total_pixels:,}")
                    print(f"   Allowed pixels: {pixels_allowed:,} ({percentage_allowed:.1f}%)")
                    print(f"   Excluded pixels: {total_pixels - pixels_allowed:,} ({100 - percentage_allowed:.1f}%)")
                    
                    if percentage_allowed < 90:
                        print(f"✅ AOI pixel clipping is working! ({100 - percentage_allowed:.1f}% pixels excluded)")
                    else:
                        print(f"⚠️  Warning: Very few pixels excluded, check AOI size")
                        
                else:
                    print(f"❌ AOI not set in masking system")
            else:
                print(f"❌ Masking system not initialized")
                
        except Exception as e:
            print(f"❌ Error processing frame: {e}")
            import traceback
            traceback.print_exc()
    else:
        print(f"❌ No 'cones' data available for testing")
else:
    print(f"❌ No 'processor' available for testing")
    print(f"💡 Create a processor first:")
    print(f"   processor = iau.SonarDataProcessor(IMAGE_PROCESSING_CONFIG)")

print(f"\n📋 Summary of key fixes made:")
print(f"   ✅ Updated SonarDataProcessor.preprocess_frame() to use new masking system")
print(f"   ✅ Added masking system initialization in constructor")
print(f"   ✅ Added AOI updates when ellipse parameters are calculated")
print(f"   ✅ Added dynamic exclusion updates from detected contours")
print(f"   ✅ Integration happens at the pixel level during edge processing")

print(f"\n🎯 The integration should now:")
print(f"   1. Create masking system automatically on first frame")
print(f"   2. Update AOI based on detected net ellipse")
print(f"   3. Prevent exclusion zones inside the AOI")
print(f"   4. Only process pixels inside the AOI (when aoi_pixel_clipping=True)")
print(f"   5. Apply these restrictions during edge detection phase")